In [4]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
import time
import kagglehub
import cv2
import pandas as pd
from sklearn.model_selection import train_test_split
import math

# Check for GPU availability
print("TensorFlow version:", tf.__version__)
print("GPU Available:", tf.config.list_physical_devices('GPU'))

# Download the ReducedMNIST dataset
path = kagglehub.dataset_download("mohamedgamal07/reduced-mnist")
print(f"Dataset downloaded to: {path}")

# List the files in the dataset
files = os.listdir(path)
print("Files in the dataset:", files)

path = path+"/Reduced MNIST Data"

train_dir = path +'/Reduced Trainging data'
test_dir = path+'/Reduced Testing data'


TensorFlow version: 2.19.0
GPU Available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Dataset downloaded to: /home/youssef-abuzeid/.cache/kagglehub/datasets/mohamedgamal07/reduced-mnist/versions/1
Files in the dataset: ['Reduced MNIST Data']


In [ ]:


# Function to load images from a directory
def load_images_from_directory(directory, n_samples_per_class=None):
    x_data, y_data = [], []
    for class_label in range(10):
        class_dir = os.path.join(directory, str(class_label))
        images = [os.path.join(class_dir, f) for f in os.listdir(class_dir) if f.endswith(('.png', '.jpg'))]
        if n_samples_per_class:
            images = np.random.choice(images, n_samples_per_class, replace=False)
        for img_path in images:
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)  # Load as grayscale
            img = img.astype('float32') / 255.0  # Normalize to [0, 1]
            x_data.append(img)
            y_data.append(class_label)
    x_data = np.array(x_data).reshape(-1, 28, 28, 1)  # Reshape for CNN
    y_data = np.array(y_data)
    return x_data, y_data

# Load training subsets
x_train_300, y_train_300 = load_images_from_directory(train_dir, n_samples_per_class=30)  # 30 per class = 300 total
x_train_700, y_train_700 = load_images_from_directory(train_dir, n_samples_per_class=70)  # 70 per class = 700 total
x_train_1000, y_train_1000 = load_images_from_directory(train_dir, n_samples_per_class=100)  # 100 per class = 1000 total

# Load test set (200 samples total)
x_test, y_test = load_images_from_directory(test_dir, n_samples_per_class=200)  

In [21]:
def random_rotate(image):
    # Generate a random float between -30 and 30 degrees
    angle_deg = tf.random.uniform([], minval=-30, maxval=30, dtype=tf.float32)

    # Convert to float and normalize for RandomRotation (±30° ≈ ±0.083 of full turn)
    angle_factor = float(angle_deg.numpy()) / 360.0

    # Apply rotation
    rotated = tf.keras.layers.RandomRotation(factor=angle_factor)(tf.expand_dims(image, 0))
    return tf.squeeze(rotated, axis=0)

def random_translate(image):
    # Random shift in pixels
    dx = tf.random.uniform([], -5, 5, dtype=tf.float32)
    dy = tf.random.uniform([], -5, 5, dtype=tf.float32)

    # Get image height and width as Python floats
    height, width = image.shape[0], image.shape[1]

    # Compute translation factor (fraction of image size)
    height_factor = float(dy.numpy()) / float(height)
    width_factor = float(dx.numpy()) / float(width)

    # Apply translation
    translated = tf.keras.layers.RandomTranslation(
        height_factor=height_factor,
        width_factor=width_factor
    )(tf.expand_dims(image, 0))

    return tf.squeeze(translated, axis=0)


def add_white_noise(image, noise_level=0.1):
    noise = tf.random.normal(shape=tf.shape(image), mean=0.0, stddev=1.0, dtype=tf.float32)
    noisy_image = image + noise * noise_level
    return tf.clip_by_value(noisy_image, 0.0, 1.0)


def augment_image(image):
    image = tf.image.convert_image_dtype(image, tf.float32)
    image = random_rotate(image)
    image = random_translate(image)
    image = add_white_noise(image)
    return image


In [22]:
# --- Controlled Augmentation Pipeline ---
def generate_dataset_with_augmentation(images, labels, num_real, num_generated):
    # Subsample real images
    real_images = images[:num_real]
    real_labels = labels[:num_real]

    if num_generated == 0:
        return real_images, real_labels

    # How many augmented samples per real sample?
    gen_per_sample = num_generated // num_real

    synthetic_images = []
    synthetic_labels = []

    for i in range(num_real):
        for _ in range(gen_per_sample):
            aug_img = augment_image(real_images[i])
            synthetic_images.append(aug_img)
            synthetic_labels.append(real_labels[i])

    synthetic_images = tf.stack(synthetic_images)
    synthetic_labels = tf.convert_to_tensor(synthetic_labels)

    # Combine real and generated
    all_images = tf.concat([real_images, synthetic_images], axis=0)
    all_labels = tf.concat([real_labels, synthetic_labels], axis=0)

    return all_images, all_labels


In [23]:
#300 samples + 1000 generated
X_train_300_1000, y_train_300_1000 = generate_dataset_with_augmentation(x_train_300, y_train_300, 300, 1000)
print("[INFO] Generated dataset with 300 real samples and 1000 synthetic samples.")
#700 samples + 1000 generated
X_train_700_1000, y_train_700_1000 = generate_dataset_with_augmentation(x_train_700, y_train_700, 700, 1000)
print("[INFO] Generated dataset with 700 real samples and 1000 synthetic samples.")
#1000 samples + 1000 generated
X_train_1000_1000, y_train_1000_1000 = generate_dataset_with_augmentation(x_train_1000, y_train_1000, 1000, 1000)
print("[INFO] Generated dataset with 1000 real samples and 1000 synthetic samples.")

#300 samples + 2000 generated
X_train_300_2000, y_train_300_2000 = generate_dataset_with_augmentation(x_train_300, y_train_300, 300, 2000)
print("[INFO] Generated dataset with 300 real samples and 2000 synthetic samples.")
#700 samples + 2000 generated
X_train_700_2000, y_train_700_2000 = generate_dataset_with_augmentation(x_train_700, y_train_700, 700, 2000)
print("[INFO] Generated dataset with 700 real samples and 2000 synthetic samples.")
#1000 samples + 2000 generated
X_train_1000_2000, y_train_1000_2000 = generate_dataset_with_augmentation(x_train_1000, y_train_1000, 1000, 2000)
print("[INFO] Generated dataset with 1000 real samples and 2000 synthetic samples.")

#300 samples + 3000 generated
X_train_300_3000, y_train_300_3000 = generate_dataset_with_augmentation(x_train_300, y_train_300, 300, 3000)
print("[INFO] Generated dataset with 300 real samples and 3000 synthetic samples.")
#700 samples + 3000 generated
X_train_700_3000, y_train_700_3000 = generate_dataset_with_augmentation(x_train_700, y_train_700, 700, 3000)
print("[INFO] Generated dataset with 700 real samples and 3000 synthetic samples.")
#1000 samples + 3000 generated
X_train_1000_3000, y_train_1000_3000 = generate_dataset_with_augmentation(x_train_1000, y_train_1000, 1000, 3000)
print("[INFO] Generated dataset with 1000 real samples and 3000 synthetic samples.")




[INFO] Generated dataset with 300 real samples and 1000 synthetic samples.
[INFO] Generated dataset with 700 real samples and 1000 synthetic samples.
[INFO] Generated dataset with 1000 real samples and 1000 synthetic samples.
[INFO] Generated dataset with 300 real samples and 2000 synthetic samples.
[INFO] Generated dataset with 700 real samples and 2000 synthetic samples.
[INFO] Generated dataset with 1000 real samples and 2000 synthetic samples.
[INFO] Generated dataset with 300 real samples and 3000 synthetic samples.
[INFO] Generated dataset with 700 real samples and 3000 synthetic samples.
[INFO] Generated dataset with 1000 real samples and 3000 synthetic samples.


In [24]:
def create_cnn_model():
    """Create a simple CNN model for digit classification"""
    model = tf.keras.Sequential([
        tf.keras.layers.Reshape((28, 28, 1), input_shape=(28, 28)),
        tf.keras.layers.Conv2D(32, kernel_size=(3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
        tf.keras.layers.Conv2D(64, kernel_size=(3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(10, activation='softmax')
    ])
    
    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model

In [25]:
model_300_000 = create_cnn_model()
model_700_000 = create_cnn_model()
model_1000_000 = create_cnn_model()

model_300_1000 = create_cnn_model()
model_700_1000 = create_cnn_model()
model_1000_1000 = create_cnn_model()

model_300_2000 = create_cnn_model()
model_700_2000 = create_cnn_model()
model_1000_2000 = create_cnn_model()

model_300_3000 = create_cnn_model()
model_700_3000 = create_cnn_model()
model_1000_3000 = create_cnn_model()


/home/youssef-abuzeid/.local/lib/python3.10/site-packages/keras/src/layers/reshaping/reshape.py:39: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


## Train the models

In [26]:
history_300_000 = model_300_000.fit(
    x_train_300, y_train_300,
    epochs=20,
    batch_size=32,    
)

from sklearn.metrics import classification_report, confusion_matrix


# Evaluate the model on the test set
y_pred_300_000 = model_300_000.predict(x_test)
y_pred_300_000_classes = np.argmax(y_pred_300_000, axis=1)
print("Classification Report for 300 samples + 0 generated:")
print(classification_report(y_test, y_pred_300_000_classes))

Epoch 1/20
 1/10 ━━━━━━━━━━━━━━━━━━━━ 19s 2s/step - accuracy: 0.1875 - loss: 2.2817

2025-04-17 01:01:17.514232: I external/local_xla/xla/service/gpu/autotuning/conv_algorithm_picker.cc:549] Omitted potentially buggy algorithm eng14{k25=0} for conv %cudnn-conv-bias-activation.6 = (f32[12,32,26,26]{3,2,1,0}, u8[0]{0}) custom-call(f32[12,1,28,28]{3,2,1,0} %bitcast.4265, f32[32,1,3,3]{3,2,1,0} %bitcast.4272, f32[32]{0} %bitcast.4738), window={size=3x3}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasActivationForward", metadata={op_type="Conv2D" op_name="sequential_12_1/conv2d_24_1/convolution" source_file="/home/youssef-abuzeid/.local/lib/python3.10/site-packages/tensorflow/python/framework/ops.py" source_line=1200}, backend_config={"operation_queue_id":"0","wait_on_operation_queues":[],"cudnn_conv_backend_config":{"conv_result_scale":1,"activation_mode":"kNone","side_input_scale":0,"leakyrelu_alpha":0},"force_earliest_schedule":false}
2025-04-17 01:01:17.563902: I external/local_xla/xla/service/gpu/autotuning/conv_algorithm_picker.cc:549] Omitted potent

10/10 ━━━━━━━━━━━━━━━━━━━━ 4s 176ms/step - accuracy: 0.1638 - loss: 2.2547
Epoch 2/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.4041 - loss: 1.9736 
Epoch 3/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5360 - loss: 1.5244 
Epoch 4/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6662 - loss: 1.0844 
Epoch 5/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7539 - loss: 0.7623 
Epoch 6/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.8182 - loss: 0.5456 
Epoch 7/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8214 - loss: 0.4924 
Epoch 8/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8736 - loss: 0.3797 
Epoch 9/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8982 - loss: 0.2977 
Epoch 10/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9168 - loss: 0.2204 
Epoch 11/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9356 - loss: 0.2091 
Epoch 12/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9339 - l

In [27]:
history_300_1000 = model_300_1000.fit(
    X_train_300_1000, y_train_300_1000,
    epochs=20,
    batch_size=32,    
)
# Evaluate the model on the test set
y_pred_300_1000 = model_300_1000.predict(x_test)
y_pred_300_1000_classes = np.argmax(y_pred_300_1000, axis=1)
print("Classification Report for 300 samples + 1000 generated:")
print(classification_report(y_test, y_pred_300_1000_classes))


Epoch 1/20
21/38 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.1377 - loss: 2.2810 

2025-04-17 01:04:37.575015: I external/local_xla/xla/service/gpu/autotuning/conv_algorithm_picker.cc:549] Omitted potentially buggy algorithm eng14{k25=0} for conv %cudnn-conv-bias-activation.6 = (f32[16,32,26,26]{3,2,1,0}, u8[0]{0}) custom-call(f32[16,1,28,28]{3,2,1,0} %bitcast.4265, f32[32,1,3,3]{3,2,1,0} %bitcast.4272, f32[32]{0} %bitcast.4738), window={size=3x3}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasActivationForward", metadata={op_type="Conv2D" op_name="sequential_15_1/conv2d_30_1/convolution" source_file="/home/youssef-abuzeid/.local/lib/python3.10/site-packages/tensorflow/python/framework/ops.py" source_line=1200}, backend_config={"operation_queue_id":"0","wait_on_operation_queues":[],"cudnn_conv_backend_config":{"conv_result_scale":1,"activation_mode":"kNone","side_input_scale":0,"leakyrelu_alpha":0},"force_earliest_schedule":false}
2025-04-17 01:04:37.611878: I external/local_xla/xla/service/gpu/autotuning/conv_algorithm_picker.cc:549] Omitted potent

38/38 ━━━━━━━━━━━━━━━━━━━━ 4s 42ms/step - accuracy: 0.1807 - loss: 2.2233
Epoch 2/20
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5343 - loss: 1.4117
Epoch 3/20
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7306 - loss: 0.8797
Epoch 4/20
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7607 - loss: 0.7036
Epoch 5/20
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8222 - loss: 0.5871
Epoch 6/20
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8698 - loss: 0.4379
Epoch 7/20
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8693 - loss: 0.3783
Epoch 8/20
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9114 - loss: 0.2882
Epoch 9/20
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9112 - loss: 0.3032
Epoch 10/20
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9297 - loss: 0.2404
Epoch 11/20
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9348 - loss: 0.2046
Epoch 12/20
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9442 - loss: 0.1672

In [28]:
history_300_2000 = model_300_2000.fit(
    X_train_300_2000, y_train_300_2000,
    epochs=20,
    batch_size=32,    
)
# Evaluate the model on the test set
y_pred_300_2000 = model_300_2000.predict(x_test)
y_pred_300_2000_classes = np.argmax(y_pred_300_2000, axis=1)
print("Classification Report for 300 samples + 2000 generated:")
print(classification_report(y_test, y_pred_300_2000_classes))


Epoch 1/20
58/66 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.2105 - loss: 2.1418

2025-04-17 01:05:11.397018: I external/local_xla/xla/service/gpu/autotuning/conv_algorithm_picker.cc:549] Omitted potentially buggy algorithm eng14{k25=0} for conv %cudnn-conv-bias-activation.6 = (f32[20,32,26,26]{3,2,1,0}, u8[0]{0}) custom-call(f32[20,1,28,28]{3,2,1,0} %bitcast.4265, f32[32,1,3,3]{3,2,1,0} %bitcast.4272, f32[32]{0} %bitcast.4738), window={size=3x3}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasActivationForward", metadata={op_type="Conv2D" op_name="sequential_18_1/conv2d_36_1/convolution" source_file="/home/youssef-abuzeid/.local/lib/python3.10/site-packages/tensorflow/python/framework/ops.py" source_line=1200}, backend_config={"operation_queue_id":"0","wait_on_operation_queues":[],"cudnn_conv_backend_config":{"conv_result_scale":1,"activation_mode":"kNone","side_input_scale":0,"leakyrelu_alpha":0},"force_earliest_schedule":false}
2025-04-17 01:05:11.435162: I external/local_xla/xla/service/gpu/autotuning/conv_algorithm_picker.cc:549] Omitted potent

66/66 ━━━━━━━━━━━━━━━━━━━━ 4s 28ms/step - accuracy: 0.2309 - loss: 2.1005
Epoch 2/20
66/66 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6192 - loss: 1.1404
Epoch 3/20
66/66 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7500 - loss: 0.7764
Epoch 4/20
66/66 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8107 - loss: 0.5855
Epoch 5/20
66/66 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8382 - loss: 0.4835
Epoch 6/20
66/66 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8825 - loss: 0.3659
Epoch 7/20
66/66 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8917 - loss: 0.3326
Epoch 8/20
66/66 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9001 - loss: 0.3165
Epoch 9/20
66/66 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9201 - loss: 0.2535
Epoch 10/20
66/66 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9154 - loss: 0.2512
Epoch 11/20
66/66 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9311 - loss: 0.2073
Epoch 12/20
66/66 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9316 - loss: 0.1960

In [29]:
history_300_3000 = model_300_3000.fit(
    X_train_300_3000, y_train_300_3000,
    epochs=20,
    batch_size=32,    
)
# Evaluate the model on the test set
y_pred_300_3000 = model_300_3000.predict(x_test)
y_pred_300_3000_classes = np.argmax(y_pred_300_3000, axis=1)
print("Classification Report for 300 samples + 3000 generated:")
print(classification_report(y_test, y_pred_300_3000_classes))
# Evaluate the model on the test set

Epoch 1/20
 99/104 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.2915 - loss: 1.9921

2025-04-17 01:05:32.004118: I external/local_xla/xla/service/gpu/autotuning/conv_algorithm_picker.cc:549] Omitted potentially buggy algorithm eng14{k25=0} for conv %cudnn-conv-bias-activation.7 = (f32[4,64,11,11]{3,2,1,0}, u8[0]{0}) custom-call(f32[4,32,13,13]{3,2,1,0} %bitcast.4800, f32[64,32,3,3]{3,2,1,0} %bitcast.4292, f32[64]{0} %bitcast.4860), window={size=3x3}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasActivationForward", metadata={op_type="Conv2D" op_name="sequential_21_1/conv2d_43_1/convolution" source_file="/home/youssef-abuzeid/.local/lib/python3.10/site-packages/tensorflow/python/framework/ops.py" source_line=1200}, backend_config={"operation_queue_id":"0","wait_on_operation_queues":[],"cudnn_conv_backend_config":{"conv_result_scale":1,"activation_mode":"kNone","side_input_scale":0,"leakyrelu_alpha":0},"force_earliest_schedule":false}


104/104 ━━━━━━━━━━━━━━━━━━━━ 4s 19ms/step - accuracy: 0.3009 - loss: 1.9690
Epoch 2/20
104/104 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7069 - loss: 0.8593
Epoch 3/20
104/104 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8209 - loss: 0.5411
Epoch 4/20
104/104 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8599 - loss: 0.4094
Epoch 5/20
104/104 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8958 - loss: 0.3208
Epoch 6/20
104/104 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9139 - loss: 0.2643
Epoch 7/20
104/104 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9302 - loss: 0.2079
Epoch 8/20
104/104 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9386 - loss: 0.1845
Epoch 9/20
104/104 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9501 - loss: 0.1494
Epoch 10/20
104/104 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9500 - loss: 0.1397
Epoch 11/20
104/104 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9552 - loss: 0.1327
Epoch 12/20
104/104 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accurac

In [30]:
history_700_000 = model_700_000.fit(

    x_train_700, y_train_700,
    epochs=20,
    batch_size=32,    
)
# Evaluate the model on the test set
y_pred_700_000 = model_700_000.predict(x_test)
y_pred_700_000_classes = np.argmax(y_pred_700_000, axis=1)
print("Classification Report for 700 samples + 0 generated:")
print(classification_report(y_test, y_pred_700_000_classes))

Epoch 1/20
18/22 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.1394 - loss: 2.2649

2025-04-17 01:06:21.117640: I external/local_xla/xla/service/gpu/autotuning/conv_algorithm_picker.cc:549] Omitted potentially buggy algorithm eng14{k25=0} for conv %cudnn-conv-bias-activation.6 = (f32[28,32,26,26]{3,2,1,0}, u8[0]{0}) custom-call(f32[28,1,28,28]{3,2,1,0} %bitcast.4265, f32[32,1,3,3]{3,2,1,0} %bitcast.4272, f32[32]{0} %bitcast.4738), window={size=3x3}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasActivationForward", metadata={op_type="Conv2D" op_name="sequential_13_1/conv2d_26_1/convolution" source_file="/home/youssef-abuzeid/.local/lib/python3.10/site-packages/tensorflow/python/framework/ops.py" source_line=1200}, backend_config={"operation_queue_id":"0","wait_on_operation_queues":[],"cudnn_conv_backend_config":{"conv_result_scale":1,"activation_mode":"kNone","side_input_scale":0,"leakyrelu_alpha":0},"force_earliest_schedule":false}
2025-04-17 01:06:21.156211: I external/local_xla/xla/service/gpu/autotuning/conv_algorithm_picker.cc:549] Omitted potent

22/22 ━━━━━━━━━━━━━━━━━━━━ 4s 88ms/step - accuracy: 0.1718 - loss: 2.2329
Epoch 2/20
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5864 - loss: 1.3815
Epoch 3/20
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7676 - loss: 0.7986
Epoch 4/20
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8247 - loss: 0.5442
Epoch 5/20
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8551 - loss: 0.4411
Epoch 6/20
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9053 - loss: 0.3374
Epoch 7/20
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9108 - loss: 0.2786
Epoch 8/20
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9200 - loss: 0.2607
Epoch 9/20
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9277 - loss: 0.2102
Epoch 10/20
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9477 - loss: 0.1683
Epoch 11/20
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9499 - loss: 0.1765
Epoch 12/20
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9455 - loss: 0.1546

In [31]:
history_700_1000 = model_700_1000.fit(
    X_train_700_1000, y_train_700_1000,
    epochs=20,
    batch_size=32,    
)
# Evaluate the model on the test set
y_pred_700_1000 = model_700_1000.predict(x_test)
y_pred_700_1000_classes = np.argmax(y_pred_700_1000, axis=1)
print("Classification Report for 700 samples + 1000 generated:")
print(classification_report(y_test, y_pred_700_1000_classes))


Epoch 1/20
44/44 ━━━━━━━━━━━━━━━━━━━━ 4s 39ms/step - accuracy: 0.2503 - loss: 2.1196
Epoch 2/20
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6345 - loss: 1.1401
Epoch 3/20
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7370 - loss: 0.8110
Epoch 4/20
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7910 - loss: 0.6682
Epoch 5/20
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8395 - loss: 0.4916
Epoch 6/20
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8437 - loss: 0.4897
Epoch 7/20
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8802 - loss: 0.3891
Epoch 8/20
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8890 - loss: 0.3464
Epoch 9/20
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8976 - loss: 0.3442
Epoch 10/20
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8915 - loss: 0.3139
Epoch 11/20
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9346 - loss: 0.2235
Epoch 12/20
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9206 - l

In [32]:
history_700_2000 = model_700_2000.fit(
    X_train_700_2000, y_train_700_2000,
    epochs=20,
    batch_size=32,    
)
# Evaluate the model on the test set
y_pred_700_2000 = model_700_2000.predict(x_test)
y_pred_700_2000_classes = np.argmax(y_pred_700_2000, axis=1)
print("Classification Report for 700 samples + 2000 generated:")
print(classification_report(y_test, y_pred_700_2000_classes))

Epoch 1/20
66/66 ━━━━━━━━━━━━━━━━━━━━ 4s 24ms/step - accuracy: 0.2365 - loss: 2.0673
Epoch 2/20
66/66 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6775 - loss: 1.0320
Epoch 3/20
66/66 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7708 - loss: 0.6993
Epoch 4/20
66/66 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8262 - loss: 0.5663
Epoch 5/20
66/66 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8521 - loss: 0.4450
Epoch 6/20
66/66 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8753 - loss: 0.3887
Epoch 7/20
66/66 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8772 - loss: 0.3554
Epoch 8/20
66/66 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9021 - loss: 0.2857
Epoch 9/20
66/66 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9178 - loss: 0.2454
Epoch 10/20
66/66 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9246 - loss: 0.2267
Epoch 11/20
66/66 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9349 - loss: 0.1857
Epoch 12/20
66/66 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9415 - l

In [33]:
history_700_3000 = model_700_3000.fit(
    X_train_700_3000, y_train_700_3000,
    epochs=20,
    batch_size=32,    
)
# Evaluate the model on the test set
y_pred_700_3000 = model_700_3000.predict(x_test)
y_pred_700_3000_classes = np.argmax(y_pred_700_3000, axis=1)
print("Classification Report for 700 samples + 3000 generated:")
print(classification_report(y_test, y_pred_700_3000_classes))


Epoch 1/20
110/110 ━━━━━━━━━━━━━━━━━━━━ 4s 13ms/step - accuracy: 0.3030 - loss: 1.9430
Epoch 2/20
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7265 - loss: 0.8556
Epoch 3/20
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8161 - loss: 0.5911
Epoch 4/20
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8539 - loss: 0.4791
Epoch 5/20
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8862 - loss: 0.3617
Epoch 6/20
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9076 - loss: 0.3037
Epoch 7/20
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9088 - loss: 0.2737
Epoch 8/20
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9241 - loss: 0.2119
Epoch 9/20
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9369 - loss: 0.1913
Epoch 10/20
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9405 - loss: 0.1650
Epoch 11/20
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9539 - loss: 0.1479
Epoch 12/20
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/ste

In [34]:
history_1000_000 = model_1000_000.fit(
    x_train_1000, y_train_1000,
    epochs=20,
    batch_size=32,    
)
# Evaluate the model on the test set
y_pred_1000_000 = model_1000_000.predict(x_test)
y_pred_1000_000_classes = np.argmax(y_pred_1000_000, axis=1)
print("Classification Report for 1000 samples + 0 generated:")
print(classification_report(y_test, y_pred_1000_000_classes))

Epoch 1/20
19/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.1879 - loss: 2.2121 

2025-04-17 01:07:42.102812: I external/local_xla/xla/service/gpu/autotuning/conv_algorithm_picker.cc:549] Omitted potentially buggy algorithm eng14{k25=0} for conv %cudnn-conv-bias-activation.7 = (f32[8,64,11,11]{3,2,1,0}, u8[0]{0}) custom-call(f32[8,32,13,13]{3,2,1,0} %bitcast.4800, f32[64,32,3,3]{3,2,1,0} %bitcast.4292, f32[64]{0} %bitcast.4860), window={size=3x3}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasActivationForward", metadata={op_type="Conv2D" op_name="sequential_14_1/conv2d_29_1/convolution" source_file="/home/youssef-abuzeid/.local/lib/python3.10/site-packages/tensorflow/python/framework/ops.py" source_line=1200}, backend_config={"operation_queue_id":"0","wait_on_operation_queues":[],"cudnn_conv_backend_config":{"conv_result_scale":1,"activation_mode":"kNone","side_input_scale":0,"leakyrelu_alpha":0},"force_earliest_schedule":false}


32/32 ━━━━━━━━━━━━━━━━━━━━ 4s 59ms/step - accuracy: 0.2577 - loss: 2.1029
Epoch 2/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6804 - loss: 0.9916
Epoch 3/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8230 - loss: 0.5873
Epoch 4/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8701 - loss: 0.4500
Epoch 5/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8857 - loss: 0.3543
Epoch 6/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9094 - loss: 0.3175
Epoch 7/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9097 - loss: 0.2956
Epoch 8/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9548 - loss: 0.1727
Epoch 9/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9541 - loss: 0.1593
Epoch 10/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9656 - loss: 0.1266
Epoch 11/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9423 - loss: 0.1812
Epoch 12/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9528 - loss: 0.1402

In [35]:
history_1000_1000 = model_1000_1000.fit(
    X_train_1000_1000, y_train_1000_1000,
    epochs=20,
    batch_size=32,    
)
# Evaluate the model on the test set
y_pred_1000_1000 = model_1000_1000.predict(x_test)
y_pred_1000_1000_classes = np.argmax(y_pred_1000_1000, axis=1)
print("Classification Report for 1000 samples + 1000 generated:")
print(classification_report(y_test, y_pred_1000_1000_classes))

Epoch 1/20
63/63 ━━━━━━━━━━━━━━━━━━━━ 4s 25ms/step - accuracy: 0.2853 - loss: 2.0294
Epoch 2/20
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7029 - loss: 0.9382
Epoch 3/20
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7960 - loss: 0.6709
Epoch 4/20
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8445 - loss: 0.5303
Epoch 5/20
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8606 - loss: 0.4266
Epoch 6/20
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8969 - loss: 0.3432
Epoch 7/20
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8847 - loss: 0.3305
Epoch 8/20
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8974 - loss: 0.2776
Epoch 9/20
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9295 - loss: 0.2282
Epoch 10/20
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9282 - loss: 0.2113
Epoch 11/20
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9275 - loss: 0.2059
Epoch 12/20
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9493 - l

In [36]:
history_1000_2000 = model_1000_2000.fit(
    X_train_1000_2000, y_train_1000_2000,
    epochs=20,
    batch_size=32,    
)
# Evaluate the model on the test set
y_pred_1000_2000 = model_1000_2000.predict(x_test)
y_pred_1000_2000_classes = np.argmax(y_pred_1000_2000, axis=1)
print("Classification Report for 1000 samples + 2000 generated:")
print(classification_report(y_test, y_pred_1000_2000_classes))


Epoch 1/20
94/94 ━━━━━━━━━━━━━━━━━━━━ 4s 18ms/step - accuracy: 0.3067 - loss: 1.9634
Epoch 2/20
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7219 - loss: 0.8406
Epoch 3/20
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8103 - loss: 0.5991
Epoch 4/20
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8396 - loss: 0.4999
Epoch 5/20
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8640 - loss: 0.4148
Epoch 6/20
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8817 - loss: 0.3636
Epoch 7/20
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9009 - loss: 0.3033
Epoch 8/20
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8969 - loss: 0.2863
Epoch 9/20
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9158 - loss: 0.2594
Epoch 10/20
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9252 - loss: 0.2387
Epoch 11/20
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9378 - loss: 0.1781
Epoch 12/20
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9407 - l

In [37]:
history_1000_3000 = model_1000_3000.fit(
    X_train_1000_3000, y_train_1000_3000,
    epochs=20,
    batch_size=32,    
)
# Evaluate the model on the test set
y_pred_1000_3000 = model_1000_3000.predict(x_test)
y_pred_1000_3000_classes = np.argmax(y_pred_1000_3000, axis=1)
print("Classification Report for 1000 samples + 3000 generated:")
print(classification_report(y_test, y_pred_1000_3000_classes))

Epoch 1/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.3409 - loss: 1.8771
Epoch 2/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7248 - loss: 0.8533
Epoch 3/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8088 - loss: 0.6059
Epoch 4/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8508 - loss: 0.4704
Epoch 5/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8788 - loss: 0.3802
Epoch 6/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8839 - loss: 0.3371
Epoch 7/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9107 - loss: 0.2784
Epoch 8/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9109 - loss: 0.2573
Epoch 9/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9338 - loss: 0.2074
Epoch 10/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9398 - loss: 0.1794
Epoch 11/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9479 - loss: 0.1776
Epoch 12/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step

(1200, 28, 28, 1)
